In [2]:
import csv
import sys
import io
import re # 정규표현식 패키지
from kiwipiepy import Kiwi, Option
import tomotopy as tp
import pandas as pd

origin_path = 'C:/Users/geun/KoreanNewsChronicle/Data/201903.csv'
result_path = "C:/Users/geun/KoreanNewsChronicle/PreProcess/201903_topic.csv"

origin=pd.read_csv(origin_path,encoding='utf-8')

text=origin['text']
print(text)

with open('C:/Users/geun/KoreanNewsChronicle/PreProcess/datas.txt','w',encoding='utf-8') as file:
    for item in text:
        file.write("%s\n" % item)

class Reader:
    def __init__(self, filePath):
        self.file = open(filePath,'r',encoding='UTF-8')
    
    def read(self, id):
        if id == 0: self.file.seek(0)
        return self.file.readline()

reader = Reader('datas.txt')

file = open("한국어불용어100.txt", 'r', encoding="utf-8")
stopword=[]

while True:
    line = file.readline()
    if not line: break
    wordlist = line.split('\t')
    if (wordlist[1].startswith('N')):
        stopword.append(wordlist[0])

print(stopword)
filename="datas.txt"  #파일 경로
file = open(filename, 'r', encoding="UTF-8")
kiwi=Kiwi()
kiwi.extract_add_words(reader.read)
kiwi.prepare()
stopwords = set(stopword)


def tokenize(sent): # 파일의 라인을 분석할 tokenize 함수
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용한다, 분석할때 나오는 결과에서 단어만 추출
    return [word
            for word, tag, _, _ in res
            if tag.startswith('N') and word not in stopwords] #불용어사전 적용


#LDA 모델을 적용해서 토픽 추출, k는 topic 개수
#alpha는 문헌-토픽 디리클레 분포의 하이퍼 파라미터
#eta는 토픽-단어 디리클레 분포의 하이퍼 파라미터 두개다 상수인듯하다
#min_cf는 단어의 최소 장서 빈도수, 전체 문헌내 출현빈도
#min_df는 단어의 최서 문헌 빈도수, 출현한 문헌 숫자수 의미
#tw는 용어 가중치 기법으로, ONE, IDF, PMI를 사용가능, ONE 보다는 PMI나 IDF 둘중 하나 사용 

file = open(result_path, "w", encoding='utf-8', newline = '')
writer = csv.writer(file)
writer.writerow(['index', 'topics'])
for i, line in enumerate(open(filename, encoding='utf-8')): #해당 경로의 파일을 받아와 한 라인씩 model에 추가
    model = tp.LDAModel(k=1, alpha=0.1, eta=0.001, min_cf=3,min_df=1, tw=tp.TermWeight.PMI)
    model.add_doc(tokenize(line)) #추출하고 모델안의 문헌을 넣는다. 즉, 학습과정에 쓰일 문헌을 생성
    model.train(0) #학습 초기화
    s = ""
    for j in range(0,100):
        model.train(20) #문헌 학습, 안에 숫자는 깁스 샘플링의 반복횟수
		#이때 기본값으로 시스템내 가용한 모든 스레드의 개수사용, 그리고 병렬화 방법을 찾아서 실행시켜준다
    for k in range(model.k): #k는 토픽의 개수
        res = model.get_topic_words(k, top_n=10) #하위 토픽 i에 해당하는 top_n개의 단어 반환
        s = ', '.join(w for w, p in res)
    writer.writerow([i, s])
    # print(i, s)
file.close()
print("End")

topic = pd.read_csv(result_path, encoding='utf-8')

# print(csv)
# print(topic['topics'])

merge = pd.concat([origin, topic['topics']], axis=1)
print(merge)
merge.to_csv('201903.csv', index=False, header=True, sep="\t")
print('merge topics')

0        美대표 “이행 점검 장·차관급 회담 年 6회 中, 합의 안 지킬 땐 관세 폭탄 즉시...
1        법원 “여성 2명·자녀 6명 송환 의무 없어” 벨기에 “10세 이하 어린이는 데려올...
2        “국제 원조 받아들이고 대선 다시 치러야”“美 인도주의적 쇼… 원조물품 일단 회수를...
3        전과자 등 총기 소유 막기 위해 의무화 상원 반대·트럼프 거부에 통과 불투명민주당이...
4        성능 실험서 배수 등 중대 결함 발견 새 장벽으로 교체 위해 8개 모두 철거미국과 ...
                               ...                        
26493    청문회서 결정적 흠결 드러나 / 잇단 부실 검증 책임 물어야 / 인사기준 강화가 우...
26494    자유한국당이 축구장 선거유세 논란에 대해 “선거관리위원회로부터 유권해석을 받은 것”...
26495    환경부가 대형마트, 백화점 등에서 1일부터 금지된 일회용 비닐봉투 사용에 대한 단속...
26496    현대모비스는 일산 킨텍스에서 열린 ‘2019 서울모터쇼’를 통해 연말까지 글로벌 최...
26497    문재인 대통령이 31일 조동호 과학기술정보통신부 장관 후보자에 대한 지명을 철회했다...
Name: text, Length: 26498, dtype: object
['것', '수', '이', '나', '사람', '등', '우리', '때', '년', '말', '일', '때문', '그것', '일', '문제', '사회', '중', '씨', '지금', '속', '하나', '집', '월', '데', '자신', '내', '경우', '명', '생각', '시간', '그녀', '앞', '번', '여자', '개', '전', '사실', '점', '정도', '원', '소리']


AttributeError: 'DataFrame' object has no attribute 'writer'